In [2]:
import pandas as pd
import os

In [4]:
DATASET_DIR = 'itunes_amazon'
train_df = pd.read_csv(os.path.join(DATASET_DIR,'train.csv'))
validation_df = pd.read_csv(os.path.join(DATASET_DIR,'valid.csv'))
test_df = pd.read_csv(os.path.join(DATASET_DIR,'test.csv'))

In [5]:
all_df = pd.concat([train_df,validation_df,test_df])
all_df.head()

,ltable_id,rtable_id,label
0,1409,51560,0
1,1289,34035,0
2,2666,35475,1
3,1157,27989,0
4,3332,7412,0


In [7]:
positives_df = all_df[all_df.label==1]
positives_df = positives_df.drop(['label'], axis=1)
positives_df.columns = ['id1','id2']

In [9]:
positives_df.to_csv('itunes_amazon/matches_itunes_amazon.csv',index=False)